In [4]:
import pandas as pd
import openpyxl
from pandas import ExcelWriter

In [5]:
%store -r tarjetas
%store -r bancos

In [ ]:
tarjeta_visa_credito = tarjetas.loc [tarjetas['Cod Marca Producto']== 'Visa'] #Filtro por la Tarjeta Visa Credito
tarjeta_visa_credito = tarjeta_visa_credito.loc[:, (tarjeta_visa_credito!= 0).any(axis=0)]

In [ ]:

       'PERCEPCION IVA R.G. 2408   1,50 %', 'PROMO CUOTAS AHORA 12/18',
       'REEMBOLSOS DE ENTIDAD', 'RETENCION IMP.GANANCIAS',
       'RETENCION ING.BRUTOS', 'RETENCION ING.BRUTOS SIRTAC', 'RETENCION IVA',
       'TOTAL DEDUCCIONES', 'TOTAL LIQUIDACION', 'VENTAS C/DESCUENTO CONTADO',
       'VENTAS C/DTO CUOTAS FINANC. OTORG.'

In [ ]:
dif = tarjeta_visa_credito.loc[tarjeta_visa_credito['Diferencia'] > 0] 

In [ ]:
dif['Diferencia']

In [6]:
tarjeta_visa_credito = tarjetas.loc [tarjetas['Cod Marca Producto']== 'Visa'] #Filtro por la Tarjeta Visa Credito
tarjeta_visa_credito = tarjeta_visa_credito.loc[:, (tarjeta_visa_credito!= 0).any(axis=0)]
tarjeta_visa_credito["PERCEPCION IVA R.G. 2408  1,50/3,00 %"] = tarjeta_visa_credito[
                    ['PERCEPCION IVA R.G. 2408   1,50 %', 'PERCEPCION IVA R.G. 2408   3,00 %']].sum(axis=1, min_count=1)
                    #sumo 2 columnas, las de percepcion de IVA
tarjeta_visa_credito['Ventas'] = tarjeta_visa_credito[['VENTAS C/DESCUENTO CONTADO',
                                'VENTAS C/DTO CUOTAS FINANC. OTORG.']].sum(axis=1, min_count=1)
                                #sumo 2 columnas, las de ventas
tarjeta_visa_credito['TOTAL DEDUCCIONES'] = tarjeta_visa_credito[['TOTAL DEDUCCIONES', 'CARGOS DE ENTIDAD']].sum(axis=1, min_count=1)
                                            #sumo 2 columnas, deducciones mas cargos de entidad
tarjeta_visa_credito['TOTAL DEDUCCIONES'] = tarjeta_visa_credito['TOTAL DEDUCCIONES'] - tarjeta_visa_credito['REEMBOLSOS DE ENTIDAD']

tarjeta_visa_credito['RETENCION ING.BRUTOS SIRTAC'] = (tarjeta_visa_credito['RETENCION ING.BRUTOS SIRTAC']
                                                    + tarjeta_visa_credito['AJUSTE SIRTAC OCT. 2DA QUIN']
                                                    - tarjeta_visa_credito['AJUSTE SIRTAC NOV. 1RA QU'])
tarjeta_visa_credito['IVA 21%'] = tarjeta_visa_credito[['IVA CRED.FISC.COMERCIO S/ARANC 21,00%', 
                                                        'IVA RI CRED.FISC.COMERCIO S/DTO F.OTORG']].sum(axis=1, min_count=1)
tarjeta_visa_credito['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'] = tarjeta_visa_credito[
                    ['DESCUENTO FINANC.OTORG. CUOTAS','PROMO CUOTAS AHORA 12/18']].sum(axis=1, min_count=1)
tarjeta_visa_credito['IVA 10.5%'] = tarjeta_visa_credito[['IVA PROMO CUOTAS AHORA 12/18 - 10,50%',
                    'IVA CRED.FISC.COM.L.25063 S/DTO F.OTOR 10,50']].sum(axis=1, min_count=1)


tarjeta_visa_credito = tarjeta_visa_credito.reindex(columns=['Fecha de Pago', 'Cod Marca Producto', 'Cod Comercio', 'Nro Liquidacion' ,'Ventas', 
'ARANCEL', 'IVA 21%','DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18', 'IVA 10.5%', 'PER B.A.I.BR.DN.01/04',
'PERC.I.B. CORRIENTES R.165/00', 'PERCEP ING BR MISIONES R.G. 003/93','PERCEPCION IVA R.G. 2408  1,50/3,00 %', 'RETENCION IMP.GANANCIAS',
'RETENCION ING.BRUTOS', 'RETENCION IVA','RETENCION ING.BRUTOS SIRTAC', 'TOTAL DEDUCCIONES' ,'TOTAL LIQUIDACION',])
#reordeno columnas, aca ya quedan en el orden que es necesario
tarjeta_visa_credito.insert(7, 'Iva 21 control', 
                          tarjeta_visa_credito['IVA 21%']/tarjeta_visa_credito['ARANCEL'])
                          #agrego columna de control para IVA 21%
tarjeta_visa_credito.insert(10, 'Iva 10.5 control', 
                          tarjeta_visa_credito['IVA 10.5%']/tarjeta_visa_credito['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'])
                          #agrego columna de control para IVA 10.5%
tarjeta_visa_credito['ARANCEL'] = tarjeta_visa_credito['IVA 21%']*100/21
tarjeta_visa_credito['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'] = tarjeta_visa_credito['IVA 10.5%']*100/10.5
tarjeta_visa_credito = tarjeta_visa_credito.assign(Diferencia = tarjeta_visa_credito.iloc[:, 5:7].sum(axis=True) 
                      + tarjeta_visa_credito.iloc[:, 8:10].sum(axis=True)
                      + tarjeta_visa_credito.iloc[:, 11:19].sum(axis=True)
                      - tarjeta_visa_credito['TOTAL DEDUCCIONES']) #agrego columna de control final


In [7]:
sum_columns = ['Nro Liquidacion', 'Ventas', 'ARANCEL', 'IVA 21%', 'Iva 21 control',
       'DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18', 'IVA 10.5%',
       'Iva 10.5 control', 'PER B.A.I.BR.DN.01/04',
       'PERC.I.B. CORRIENTES R.165/00', 'PERCEP ING BR MISIONES R.G. 003/93',
       'PERCEPCION IVA R.G. 2408  1,50/3,00 %', 'RETENCION IMP.GANANCIAS',
       'RETENCION ING.BRUTOS', 'RETENCION IVA', 'RETENCION ING.BRUTOS SIRTAC',
       'TOTAL DEDUCCIONES', 'TOTAL LIQUIDACION', 'Diferencia']
max_columns = ['Nro Liquidacion']

group = {col: ('max' if col in max_columns else 'sum') for col in sum_columns}

tarjeta_visa_credito=tarjeta_visa_credito.groupby(by=['Cod Marca Producto', 'Cod Comercio']).agg(group)

In [9]:
tarjeta_visa_credito = tarjeta_visa_credito.drop(['Diferencia', 'Iva 21 control', 'Iva 10.5 control'], axis=1)

In [10]:
tarjeta_visa_credito = tarjeta_visa_credito.reset_index()
tarjeta_visa_credito = tarjeta_visa_credito.merge(bancos)

In [12]:
writer = ExcelWriter(r'C:\Users\Juan M Cabral\Desktop\Liquidaciones tarjetas\11-22\Visadecredito1122.xlsx')
tarjeta_visa_credito.to_excel(writer, 'Hoja de datos', index=False)
writer.save()

C:\Users\Juan M Cabral\AppData\Local\Temp\ipykernel_10312\4051872596.py:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [8]:
tarjeta_visa_credito

Desc Concepto Clearing           Nro Liquidacion       Ventas       ARANCEL  \
Cod Marca Producto Cod Comercio                                               
Visa               25098024               149218  24719665.03  4.562726e+05   
                   25107096               149071   5247160.85  9.797433e+04   
                   25107105               149219  11898986.56  2.235653e+05   
                   25107118               149220  10552141.90  1.922348e+05   
                   25107120               149221  10028859.27  1.856441e+05   
                   25107146               149222   6617824.97  1.390818e+05   
                   25107159               148405   4638960.75  8.350081e+04   
                   25107161               149223  58549780.13  1.065185e+06   
                   25107174               149224  26162754.98  4.747785e+05   
                   25107187               149225  41268243.22  7.550637e+05   
                   25107191               114668   4539721.41  8.848057e+04   
                   25107209               114669   5902187.47  1.062387e+05   
                   25107211               149079  10591884.64  2.019287e+05   
                   25107224               149226   5427376.09  1.020980e+05   
                   25107241               149227  13393960.29  2.681724e+05   
                   25107252               149228   3216212.93  7.529224e+04   
                   25107265               148804   4458208.69  1.047820e+05   
                   25107278               149229   6011574.07  1.245301e+05   
                   25107280               149230   3853743.73  8.432057e+04   
                   25107293               149231   7570183.46  1.518311e+05   
                   25107315               149232  10289785.59  2.900240e+05   
                   25107328               149233  14581171.08  2.724450e+05   
                   25107330               149234   7875777.50  1.417626e+05   
                   25107343               149235   5163246.10  9.462886e+04   
                   25107356               149236  18347473.00  3.504227e+05   
                   25107369               149237  13794886.18  2.483060e+05   
                   25107371               149238  30733254.21  5.777218e+05   
                   25107384               149239  10109756.12  1.845028e+05   
                   25107397               149240   9332044.22  1.744203e+05   
                   25107406                24553  10192356.54  2.052931e+05   
                   25107419               149241  10168327.43  2.003455e+05   
                   25107421               148666   4778810.48  8.601795e+04   
                   25107434               148521   4649786.28  8.760595e+04   
                   25107447               148809  15127775.24  2.793524e+05   
                   25107462                24510   2304657.57  4.242710e+04   
                   25107488               149242  15022205.58  2.971139e+05   
                   25107501               149094   5490811.00  1.025370e+05   
                   25107512               149095   5884221.21  1.179986e+05   
                   25107525               149243  14118208.07  2.671189e+05   
                   25107538                24470   3433045.53  6.525410e+04   
                   25107540               149244   2788281.87  5.018862e+04   
                   25107553               149245  13465426.36  2.437289e+05   
                   25107566               149246   9673052.61  2.010181e+05   
                   25107579               149247   4422205.03  9.098943e+04   
                   25107581               149098   4816826.79  9.206810e+04   
                   25107594               149248   6620255.10  1.234100e+05   
                   25107603               149249   3924084.24  7.063314e+04   
                   25107616                24554   3185276.02  6.017576e+04   
                   25107629                52915  